# Human Resources Department Attrition Analysis
In this analysis, we explore the data given to us to help understand the current attrition rate of the company.

In [19]:
# Load the appropriate libraries
shh <- suppressPackageStartupMessages
shh(library(ggplot2))
shh(library(caret))

In [20]:
# Load the CSV file with employee data.
hr_data <- read.csv('https://bit.ly/2VTrfPs')

# The next four data sets will preview basic knowledge about the the overall data.

In [2]:
# Calculate the dimensions of the dataset.
dim(hr_data)

[1] 1470   25

In [3]:
# Preview the first few records in the dataset.
head(hr_data)

Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6


In [5]:
# List and describe each of the fields in the dataset.
str(hr_data)

'data.frame':	1470 obs. of  25 variables:
 $ Attrition              : int  1 0 1 0 0 0 0 0 0 0 ...
 $ EmployeeNumber         : int  1 2 4 5 7 8 10 11 12 13 ...
 $ Age                    : int  41 49 37 33 27 32 59 30 38 36 ...
 $ Gender                 : Factor w/ 2 levels "Female","Male": 1 2 2 1 2 2 1 2 2 2 ...
 $ MaritalStatus          : Factor w/ 3 levels "Divorced","Married",..: 3 2 3 2 2 3 2 1 3 2 ...
 $ Department             : Factor w/ 3 levels "Human Resources",..: 3 2 2 2 2 2 2 2 2 2 ...
 $ JobRole                : Factor w/ 9 levels "Healthcare Representative",..: 8 7 3 7 3 3 3 3 5 1 ...
 $ MonthlyIncome          : int  5993 5130 2090 2909 3468 3068 2670 2693 9526 5237 ...
 $ OverTime               : Factor w/ 2 levels "No","Yes": 2 1 2 2 1 1 2 1 1 1 ...
 $ EducationField         : Factor w/ 6 levels "Human Resources",..: 2 2 5 2 4 2 4 2 2 4 ...
 $ BusinessTravel         : Factor w/ 3 levels "Non-Travel","Travel_Frequently",..: 3 2 3 2 3 2 3 3 2 3 ...
 $ DistanceFromHome   

In [6]:
# Calculate descriptive stats for each of the fields in the dataset.
summary(hr_data)

   Attrition      EmployeeNumber        Age           Gender     MaritalStatus
 Min.   :0.0000   Min.   :   1.0   Min.   :18.00   Female:588   Divorced:327  
 1st Qu.:0.0000   1st Qu.: 491.2   1st Qu.:30.00   Male  :882   Married :673  
 Median :0.0000   Median :1020.5   Median :36.00                Single  :470  
 Mean   :0.1612   Mean   :1024.9   Mean   :36.92                              
 3rd Qu.:0.0000   3rd Qu.:1555.8   3rd Qu.:43.00                              
 Max.   :1.0000   Max.   :2068.0   Max.   :60.00                              
                                                                              
                  Department                       JobRole    MonthlyIncome  
 Human Resources       : 63   Sales Executive          :326   Min.   : 1009  
 Research & Development:961   Research Scientist       :292   1st Qu.: 2911  
 Sales                 :446   Laboratory Technician    :259   Median : 4919  
                              Manufacturing Director   :

# Below are some visual charts that will show possible major reasons for attrition.

In [23]:
ggplot(hr_data, aes(x=Attrition, y=Age)) + 
    geom_line() + (aes(color = Gender))

In [33]:
ggplot(hr_data, aes(x = Gender, fill = Attrition)) + 
    geom_density() + (aes(color = Gender)) 

In [38]:
ggplot(hr_data, aes(x = MonthlyIncome, fill = Attrition)) + 
    geom_dotplot() + (aes(color = Gender))

`stat_bindot()` using `bins = 30`. Pick better value with `binwidth`.


In [39]:
# Evaluates whether or not there are missing/null values and addresses those, if they exist.
sum(is.na(hr_data))

[1] 0

In [41]:
# Allows to set the random number generator to the same.
set.seed(3456)  
trainIndex <- createDataPartition(hr_data$Attrition, p = .8, list = FALSE, times = 1)
hr_train <- hr_data[ trainIndex,] 
hr_test <- hr_data[-trainIndex,]

In [43]:
# Try to build a regression model with all variables.
hr_train_model <- glm (Attrition ~ Age + Gender + MaritalStatus + MonthlyIncome, 
                          data=hr_train, family=binomial)

summary(hr_train_model)


Call:
glm(formula = Attrition ~ Age + Gender + MaritalStatus + MonthlyIncome, 
    family = binomial, data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.1450  -0.6253  -0.4929  -0.2987   2.7028  

Coefficients:
                       Estimate Std. Error z value Pr(>|z|)    
(Intercept)          -7.442e-01  4.282e-01  -1.738 0.082244 .  
Age                  -3.137e-02  1.098e-02  -2.856 0.004294 ** 
GenderMale            3.057e-01  1.746e-01   1.750 0.080050 .  
MaritalStatusMarried  3.146e-01  2.501e-01   1.258 0.208392    
MaritalStatusSingle   1.029e+00  2.481e-01   4.148 3.35e-05 ***
MonthlyIncome        -9.775e-05  2.768e-05  -3.531 0.000414 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 1023.56  on 1175  degrees of freedom
Residual deviance:  942.48  on 1170  degrees of freedom
AIC: 954.48

Number of Fisher Scoring iterations: 5


In [45]:
# Assess the importance of each variable.
varImp(hr_train_model)

,Overall
,<dbl>
Age,2.855730
GenderMale,1.750394
MaritalStatusMarried,1.257999
MaritalStatusSingle,4.148028
MonthlyIncome,3.531139


In [51]:
# Add and populate the "Attrition" prediction in the test dataset.
hr_test$Attrition_prediction <- predict(hr_train_model,newdata=hr_test,type='response')
head(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_prediction
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
6,0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,13,0,2,8,0,7,7,3,6,0.3289514
8,0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,22,1,2,1,1,1,0,0,0,0.1620813
11,0,14,35,Male,Married,Research & Development,Laboratory Technician,2426,No,Medical,...,13,1,5,6,0,5,4,0,3,0.1886208
17,0,21,32,Male,Divorced,Research & Development,Research Scientist,3298,Yes,Life Sciences,...,12,2,5,7,0,6,2,0,5,0.1462027
22,1,27,36,Male,Single,Sales,Sales Representative,3407,No,Life Sciences,...,23,0,4,10,7,5,3,0,3,0.2949338
23,0,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,11,0,4,13,0,12,6,2,11,0.1241285


In [49]:
# Write the "Attrition" preductions to a new file.
write.csv(hr_test, "Attrition_test_predictions.csv", row.names=FALSE)

# Based on your analysis/research, how do you propose minimizing turnover?

# It seems to me that majority of the people who are planning to change work or who are changing work are Male workers between the age of 28-40. We do not want to call it a mid-life crisis problem, but it seems to be that these employees, lack satisfaction or fulfillment in their current jobs.

# Recommendation: I believe that if the HR department can come up with pro-engagement activities or motivational workshops more often, we are going to be able to pull them back to be a more productive and fulfilled employee.